In [1]:
library(nichenetr)
library(Seurat) # please update to Seurat V4
library(tidyverse)

Attaching SeuratObject

── Attaching packages ────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [12]:
# seuratObj = readRDS('data/seuratObj.rds')
# # seuratObj@meta.data = seuratObj@meta.data %>% 
# #     dplyr::rename(Idents = celltype)
# head(seuratObj@meta.data)

,nGene,nUMI,orig.ident,aggregate,res.0.6,celltype,nCount_RNA,nFeature_RNA
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>
W380370,880,1611,LN_SS,SS,1,CD8 T,1607,876
W380372,541,891,LN_SS,SS,0,CD4 T,885,536
W380374,742,1229,LN_SS,SS,0,CD4 T,1223,737
W380378,847,1546,LN_SS,SS,1,CD8 T,1537,838
W380379,839,1606,LN_SS,SS,0,CD4 T,1603,836
W380381,517,844,LN_SS,SS,0,CD4 T,840,513


In [8]:
seuratObj

An object of class Seurat 
13541 features across 5027 samples within 1 assay 
Active assay: RNA (13541 features, 1575 variable features)
 4 dimensional reductions calculated: cca, cca.aligned, tsne, pca

In [3]:
ligand_target_matrix = readRDS('data/ligand_target_matrix.rds')
ligand_target_matrix[1:5,1:5]

,CXCL1,CXCL2,CXCL3,CXCL5,PPBP
A1BG,3.534343e-04,4.041324e-04,3.729920e-04,3.080640e-04,2.628388e-04
A1BG-AS1,1.650894e-04,1.509213e-04,1.583594e-04,1.317253e-04,1.231819e-04
A1CF,5.787175e-04,4.596295e-04,3.895907e-04,3.293275e-04,3.211944e-04
A2M,6.027058e-04,5.996617e-04,5.164365e-04,4.517236e-04,4.590521e-04
A2M-AS1,8.898724e-05,8.243341e-05,7.484018e-05,4.912514e-05,5.120439e-05


In [4]:
lr_network = readRDS('data/lr_network.rds')
head(lr_network)

from,to,source,database
<chr>,<chr>,<chr>,<chr>
CXCL1,CXCR2,kegg_cytokines,kegg
CXCL2,CXCR2,kegg_cytokines,kegg
CXCL3,CXCR2,kegg_cytokines,kegg
CXCL5,CXCR2,kegg_cytokines,kegg
PPBP,CXCR2,kegg_cytokines,kegg
CXCL6,CXCR2,kegg_cytokines,kegg


In [6]:
weighted_networks = readRDS('data/weighted_networks.rds')
weighted_networks_lr = weighted_networks$lr_sig %>% 
    inner_join(lr_network %>% 
               distinct(from,to), 
               by = c("from","to"))
head(weighted_networks$lr_sig)

from,to,weight
<chr>,<chr>,<dbl>
A1BG,ABCC6,0.42164389
A1BG,ACE2,0.10074109
A1BG,ADAM10,0.09698978
A1BG,AGO1,0.05245879
A1BG,AKT1,0.08553468
A1BG,ANXA7,0.45659947


## Define the sender and receiver niches

In [13]:
## receiver
receiver = "CD8+ T cells"
# receiver = "CD8 T"
expressed_genes_receiver = get_expressed_genes(receiver, seuratObj, pct = 0.10)

background_expressed_genes = expressed_genes_receiver %>% 
    .[. %in% rownames(ligand_target_matrix)]

In [ ]:
## sender
sender_celltypes = c("CD4+ T cells",
                     "Dendritic cells", 
                     "Regulatory T cells", 
                     "B cells", 
                     "Macrophages")

list_expressed_genes_sender = sender_celltypes %>% 
    unique() %>% 
# lapply to get the expressed genes of every sender cell type separately here
    lapply(get_expressed_genes, seuratObj, 0.10) 
expressed_genes_sender = list_expressed_genes_sender %>% 
    unlist() %>% 
    unique()

## Define a gene set of interest

In [15]:
seurat_obj_receiver = subset(seuratObj, 
                             idents = receiver)
# head(seurat_obj_receiver)
seurat_obj_receiver = SetIdent(seurat_obj_receiver, 
                               value = seurat_obj_receiver[["aggregate"]])

condition_oi = "LCMV"
condition_reference = "SS" 
  
DE_table_receiver = FindMarkers(
    object = seurat_obj_receiver, 
    ident.1 = condition_oi, 
    ident.2 = condition_reference, 
    min.pct = 0.10
) %>% 
    rownames_to_column("gene")

geneset_oi = DE_table_receiver %>% 
    filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25) %>% 
    pull(gene)
geneset_oi = geneset_oi %>% 
    .[. %in% rownames(ligand_target_matrix)]

,nGene,nUMI,orig.ident,aggregate,res.0.6,celltype,nCount_RNA,nFeature_RNA
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>
W380370,880,1611,LN_SS,SS,1,CD8 T,1607,876
W380378,847,1546,LN_SS,SS,1,CD8 T,1537,838
W380386,1336,3132,LN_SS,SS,1,CD8 T,3118,1325
W380393,1130,2377,LN_SS,SS,1,CD8 T,2369,1122
W380395,948,1719,LN_SS,SS,1,CD8 T,1711,941
W380401,829,1692,LN_SS,SS,1,CD8 T,1686,826
W380410,909,1797,LN_SS,SS,1,CD8 T,1788,900
W380417,1125,2296,LN_SS,SS,1,CD8 T,2289,1118
W380418,744,1356,LN_SS,SS,1,CD8 T,1353,741
